https://colab.research.google.com/github/eugenesiow/practical-ml/blob/master/notebooks/Sentiment_Analysis_Movie_Reviews.ipynb#scrollTo=z1NgMDgTIBsh

In [1]:
# работа с данными
import pandas as pd
import numpy as np

# графики
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

# настройка графиков
# matplotlib.rcParams['figure.figsize'] = (8, 8)
# sns.set_style('whitegrid')

# предупреждения
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

# tqdm
from tqdm.notebook import tqdm
# time
import itertools
# copy
from copy import deepcopy

# работа с файлами
import pickle
import json

In [10]:
#unlabeled = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/unlabeled_data.csv")
train  = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/train.csv")
test = pd.read_csv("C:/Users/ranas/ML course/Competitions/Second competition/data/raw/test.csv")

In [11]:
import sys
sys.path.insert(1, 'C:/Users/ranas/ML course/Competitions/Second competition/functions')

In [35]:
from bs4 import BeautifulSoup
import re

train['tidy_review'] = train['review']

for j in tqdm(range(train.shape[0])):
    # берем один отзыв
    text = train['review'][j]
    # все слова к одному регистру
    # text = text.lower()
    # html
    text = BeautifulSoup(text, 'lxml').text
    # ненужные символы и знаки
    text = re.sub('[+\-*/]', '', text)
    text = re.sub('\d+(\.\d*)?', '', text)
    train['tidy_review'][j] = text


  0%|          | 0/12500 [00:00<?, ?it/s]

C:\Users\ranas\AppData\Local\Temp\ipykernel_2632\2763909700.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['tidy_review'][j] = text
C:\Users\ranas\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [37]:
train.head()

,id,sentiment,review,tidy_review
0,c77111d1e3526ac0f7cf13af46b65e65,1,I thought the film could be a bit more complex...,I thought the film could be a bit more complex...
1,c4806c50f6560aa65f15ed69a25940f2,0,...except for Jon Heder. This guy tanked the e...,...except for Jon Heder. This guy tanked the e...
2,82b113c68594c74ab8dbe630206ec28a,1,"Jim Carrey is good as usual, and even though t...","Jim Carrey is good as usual, and even though t..."
3,3d9891905de05248dfc2550239bd4687,1,Just read the original story which is written ...,Just read the original story which is written ...
4,da8d5a23f01e129b534022149825caa2,1,I watched this film over a hundred times. It i...,I watched this film over a hundred times. It i...


In [45]:
train_df = train.drop(['id', 'review'], axis=1)
train_df.columns = ['labels', 'text']
train_df.head()

0        1
1        0
2        1
3        1
4        1
        ..
12495    0
12496    1
12497    1
12498    1
12499    0
Name: labels, Length: 12500, dtype: int64

In [48]:
train_df.text

0        I thought the film could be a bit more complex...
1        ...except for Jon Heder. This guy tanked the e...
2        Jim Carrey is good as usual, and even though t...
3        Just read the original story which is written ...
4        I watched this film over a hundred times. It i...
                               ...                        
12495    Distasteful, cliched thriller has young couple...
12496    Stewart is a Wyoming cattleman who dreams to m...
12497     is in many ways a \lost year\" in motion pict...
12498    I felt this film  throughout. I waas impressed...
12499    the intention the directors has for this films...
Name: text, Length: 12500, dtype: object

In [39]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'learning_rate': 0.00001,
    'weight_decay': 0.01,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [50]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=2, args=train_args, use_cuda=False) 

# Train the model, there is no development or validation set for this dataset 
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /xlnet-base-cased/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['lo

  0%|          | 0/12500 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 80368 features created from 12500 samples.


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/628 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
samples = ['The script is nice.Though the casting is absolutely non-watchable.No style. the costumes do not look like some from the High Highbury society. Comparing Gwyneth Paltrow with Kate Beckinsale I can only say that Ms. Beckinsale speaks British English better than Ms. Paltrow, though in Ms. Paltrow\'s acting lies the very nature of Emma Woodhouse. Mr. Northam undoubtedly is the best Mr. Knightley of all versions, he is romantic and not at all sharp-looking and unfeeling like Mr. Knightley in the TV-version. P.S.The spectator cannot see at all Mr. Elton-Ms. Smith relationship\'s development as it was in the motion version, so one cannot understand where was all Emma\'s trying of make a Elton-Smith match (besides of the portrait).']
predictions, _ = model.predict(samples)
label_dict = {0: 'negative', 1: 'positive'}
for idx, sample in enumerate(samples):
  print('{} - {}: {}'.format(idx, label_dict[predictions[idx]], sample))